In [39]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np

In [40]:
#Now we define a new dataframe called data from yfinance which will contain the stock price of apple on a specified date range.
# Define the stock ticker and date range
ticker = 'AAPL'
start_date = '2011-05-01'
end_date = '2024-11-20'

# Fetch the data using yfinance
data = yf.download(ticker, start=start_date, end=end_date)

# Display the DataFrame
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-05-02,12.490714,12.516786,12.339286,12.367143,10.432265,442713600
2011-05-03,12.428214,12.496071,12.343571,12.435714,10.490109,313348000
2011-05-04,12.437857,12.565357,12.388571,12.484643,10.531385,389250400
2011-05-05,12.442857,12.533929,12.358929,12.383929,10.446430,335969200
2011-05-06,12.488929,12.500000,12.364643,12.380714,10.443715,280134400
...,...,...,...,...,...,...
2024-11-13,224.009995,226.649994,222.759995,225.119995,225.119995,48566200
2024-11-14,225.020004,228.869995,225.000000,228.220001,228.220001,44923900
2024-11-15,226.399994,226.919998,224.270004,225.000000,225.000000,47923700


In [41]:
import numpy as np
data['VWAP'] = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()

# Calculate the log difference for 'Open', 'Close', 'High', 'Low', 'Volume'
columns_to_calculate = ['Open', 'Close', 'High', 'Low', 'Volume', 'VWAP']
for column in columns_to_calculate:
    data[f'log_diff_{column}'] = np.log(data[column] / data[column].shift(1))

data.loc[:, 'Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)


# List of columns to calculate logarithms for
columns_to_log = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'VWAP']

# Ensure the columns exist in the DataFrame before processing
existing_columns = [col for col in columns_to_log if col in data.columns]

# Add new columns for logarithms
for column in existing_columns:
    data[f'log_{column}'] = np.log(data[column])

In [42]:
horizons = [2,5,60,250,1000]
new_predictors = []
data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
for horizon in horizons:
    rolling_averages = data.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    data[ratio_column] = data["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    data[trend_column] = data.shift(1).rolling(horizon).sum()["Target"]
    data[f'Volatility_{horizon}'] = data['Log_Returns'].rolling(window=horizon).std()

    new_predictors+= [ratio_column, trend_column]

# Ensure 'Close' contains the daily closing prices


# Calculate rolling volatility (e.g., 20-day rolling window)
# Remove rows with any null values
data = data.dropna()
data

,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,log_diff_Close,log_diff_High,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
Date,,,,,,,,,,,,,,,,,,,,,
2015-04-23,32.075001,32.605000,32.035000,32.417500,28.990734,183083600,18.820088,0.010263,0.008130,0.011956,...,0.013813,1.032983,35.0,0.014629,1.217182,138.0,0.013436,1.636167,521.0,0.016971
2015-04-24,32.622501,32.657501,32.307499,32.570000,29.127119,178103600,18.826237,0.016925,0.004693,0.001609,...,0.010376,1.035784,35.0,0.012935,1.220827,138.0,0.013240,1.642195,521.0,0.016971
2015-04-27,33.077499,33.282501,32.787498,33.162498,29.656992,387816800,18.840184,0.013851,0.018028,0.018957,...,0.008950,1.052708,35.0,0.012558,1.240802,139.0,0.013277,1.670328,521.0,0.016979
2015-04-28,33.615002,33.634998,32.392502,32.639999,29.189713,475696000,18.856631,0.016119,-0.015881,0.010535,...,0.013074,1.034289,35.0,0.012584,1.219142,139.0,0.013319,1.642335,521.0,0.016985
2015-04-29,32.540001,32.897499,32.075001,32.160000,28.760462,253544400,18.865077,-0.032502,-0.014815,-0.022171,...,0.014873,1.017734,34.0,0.012685,1.199236,138.0,0.013360,1.616574,521.0,0.016993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13,224.009995,226.649994,222.759995,225.119995,225.119995,48566200,43.073718,-0.002408,0.003961,0.004688,...,0.012161,0.992898,30.0,0.012615,1.120413,134.0,0.014121,1.361441,525.0,0.016944
2024-11-14,225.020004,228.869995,225.000000,228.220001,228.220001,44923900,43.086048,0.004499,0.013677,0.009747,...,0.009385,1.006436,31.0,0.012739,1.134972,134.0,0.014135,1.379234,526.0,0.016921
2024-11-15,226.399994,226.919998,224.270004,225.000000,225.000000,47923700,43.098971,0.006114,-0.014210,-0.008557,...,0.011576,0.992201,31.0,0.012828,1.118173,134.0,0.014167,1.358872,525.0,0.016924


In [43]:
features = ['Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000']
target = ['Target']

In [44]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `data` is your DataFrame
features = features
target = ['Target']

# Split data into training (80%) and testing (20%) sets
split_index = int(len(data) * 0.8)
train_data = data.iloc[:split_index]
test_data = data.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target].values.ravel()
X_test = test_data[features]
y_test = test_data[target].values.ravel()

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.5  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


Accuracy: 0.51

Confusion Matrix:
[[124  95]
 [143 121]]

Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.57      0.51       219
           1       0.56      0.46      0.50       264

    accuracy                           0.51       483
   macro avg       0.51      0.51      0.51       483
weighted avg       0.52      0.51      0.51       483


Feature Importances:
             Feature  Importance
3      Close_Ratio_5    0.098003
0        Log_Returns    0.089560
1      Close_Ratio_2    0.089408
5     Close_Ratio_60    0.088058
8    Close_Ratio_250    0.087809
13   Volatility_1000    0.086766
11  Close_Ratio_1000    0.084880
7      Volatility_60    0.084356
10    Volatility_250    0.083003
12        Trend_1000    0.057262
9          Trend_250    0.052138
6           Trend_60    0.049616
4            Trend_5    0.031261
2            Trend_2    0.017879


In [45]:
# Reset the index to make 'Date' a column
data = data.reset_index()

# Extract only the date part and convert to string format
data['Date'] = data['Date'].dt.date.astype(str)
data

,Date,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,log_diff_Close,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
0,2015-04-23,32.075001,32.605000,32.035000,32.417500,28.990734,183083600,18.820088,0.010263,0.008130,...,0.013813,1.032983,35.0,0.014629,1.217182,138.0,0.013436,1.636167,521.0,0.016971
1,2015-04-24,32.622501,32.657501,32.307499,32.570000,29.127119,178103600,18.826237,0.016925,0.004693,...,0.010376,1.035784,35.0,0.012935,1.220827,138.0,0.013240,1.642195,521.0,0.016971
2,2015-04-27,33.077499,33.282501,32.787498,33.162498,29.656992,387816800,18.840184,0.013851,0.018028,...,0.008950,1.052708,35.0,0.012558,1.240802,139.0,0.013277,1.670328,521.0,0.016979
3,2015-04-28,33.615002,33.634998,32.392502,32.639999,29.189713,475696000,18.856631,0.016119,-0.015881,...,0.013074,1.034289,35.0,0.012584,1.219142,139.0,0.013319,1.642335,521.0,0.016985
4,2015-04-29,32.540001,32.897499,32.075001,32.160000,28.760462,253544400,18.865077,-0.032502,-0.014815,...,0.014873,1.017734,34.0,0.012685,1.199236,138.0,0.013360,1.616574,521.0,0.016993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,2024-11-13,224.009995,226.649994,222.759995,225.119995,225.119995,48566200,43.073718,-0.002408,0.003961,...,0.012161,0.992898,30.0,0.012615,1.120413,134.0,0.014121,1.361441,525.0,0.016944
2408,2024-11-14,225.020004,228.869995,225.000000,228.220001,228.220001,44923900,43.086048,0.004499,0.013677,...,0.009385,1.006436,31.0,0.012739,1.134972,134.0,0.014135,1.379234,526.0,0.016921
2409,2024-11-15,226.399994,226.919998,224.270004,225.000000,225.000000,47923700,43.098971,0.006114,-0.014210,...,0.011576,0.992201,31.0,0.012828,1.118173,134.0,0.014167,1.358872,525.0,0.016924
2410,2024-11-18,225.250000,229.740005,225.169998,228.020004,228.020004,44686000,43.111219,-0.005092,0.013333,...,0.011470,1.005432,31.0,0.012876,1.132358,134.0,0.014179,1.376181,525.0,0.016927


In [46]:
df2 = pd.read_csv("stock_news_sentiment_analysis_results_AAPL.csv")
df3 = df2.drop(['Ticker'],axis=1)
df = df3.drop(['weighted positive sentiment score', 'weighted negative sentiment score', 'weighted neutral sentiment score'],axis=1)
df

,Date,weighted compound sentiment score
0,2011-06-07,-0.700300
1,2011-06-14,0.296000
2,2011-06-20,-0.128000
3,2011-07-07,0.474000
4,2011-07-13,0.921700
...,...,...
1992,2024-11-05,0.317723
1993,2024-11-06,0.223974
1994,2024-11-07,0.375573
1995,2024-11-08,0.462160


In [47]:
# Merge df and data on the 'Date' column
# This performs a left join, keeping all rows from df and adding matching rows from data
df = pd.merge(df, data, on='Date', how='left')

# Display the modified df
df

,Date,weighted compound sentiment score,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
0,2011-06-07,-0.700300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-06-14,0.296000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-06-20,-0.128000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-07-07,0.474000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-07-13,0.921700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,2024-11-05,0.317723,221.800003,223.949997,221.139999,223.449997,223.204422,28111300.0,43.001952,0.003659,...,0.010144,0.985864,33.0,0.012535,1.117414,136.0,0.014140,1.356573,526.0,0.016938
1993,2024-11-06,0.223974,222.610001,226.070007,221.190002,222.720001,222.475235,54561100.0,43.016493,0.003645,...,0.009654,0.982538,32.0,0.012352,1.112877,135.0,0.014138,1.351293,525.0,0.016939
1994,2024-11-07,0.375573,224.630005,227.880005,224.570007,227.479996,227.229996,42137700.0,43.028020,0.009033,...,0.013088,1.003112,32.0,0.012646,1.135638,136.0,0.014195,1.379276,525.0,0.016949
1995,2024-11-08,0.462160,227.169998,228.660004,226.410004,226.960007,226.960007,38328800.0,43.038473,0.011244,...,0.010683,1.000655,31.0,0.012534,1.132125,135.0,0.014135,1.375226,525.0,0.016948


In [48]:
df = df.dropna()

In [49]:
df

,Date,weighted compound sentiment score,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
153,2015-04-27,0.000000,33.077499,33.282501,32.787498,33.162498,29.656992,387816800.0,18.840184,0.013851,...,0.008950,1.052708,35.0,0.012558,1.240802,139.0,0.013277,1.670328,521.0,0.016979
154,2015-05-14,-0.128000,31.852501,32.237499,31.790001,32.237499,28.950195,180814000.0,18.939399,0.009939,...,0.014410,1.014854,29.0,0.013945,1.181761,137.0,0.013715,1.603188,521.0,0.017047
155,2015-05-20,-0.381800,32.500000,32.744999,32.334999,32.514999,29.199400,145819600.0,18.962260,-0.005294,...,0.010664,1.024131,28.0,0.013511,1.184424,134.0,0.013715,1.610431,520.0,0.017041
156,2015-06-11,-0.648600,32.294998,32.544998,32.119999,32.147499,28.869362,141563600.0,19.047266,0.009802,...,0.007260,1.006298,26.0,0.012512,1.146872,132.0,0.013670,1.568224,521.0,0.016991
157,2015-06-16,0.401900,31.757500,31.962500,31.592501,31.900000,28.647106,125976400.0,19.061404,0.007348,...,0.008518,0.998574,26.0,0.012385,1.133786,131.0,0.013685,1.551575,520.0,0.016948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,2024-11-04,0.269451,220.990005,222.789993,219.710007,222.009995,221.766006,44944500.0,42.994428,0.000091,...,0.008224,0.979937,33.0,0.012531,1.111138,136.0,0.014161,1.348685,525.0,0.016938
1992,2024-11-05,0.317723,221.800003,223.949997,221.139999,223.449997,223.204422,28111300.0,43.001952,0.003659,...,0.010144,0.985864,33.0,0.012535,1.117414,136.0,0.014140,1.356573,526.0,0.016938
1993,2024-11-06,0.223974,222.610001,226.070007,221.190002,222.720001,222.475235,54561100.0,43.016493,0.003645,...,0.009654,0.982538,32.0,0.012352,1.112877,135.0,0.014138,1.351293,525.0,0.016939
1994,2024-11-07,0.375573,224.630005,227.880005,224.570007,227.479996,227.229996,42137700.0,43.028020,0.009033,...,0.013088,1.003112,32.0,0.012646,1.135638,136.0,0.014195,1.379276,525.0,0.016949


In [50]:
features = ['weighted compound sentiment score','Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000']
target = ['Target']
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `df` is your DataFrame
features = features
target = 'Target'

# Split data into training (80%) and testing (20%) sets
split_index = int(len(df) * 0.8)
train_data = df.iloc[:split_index]
test_data = df.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.5  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


Accuracy: 0.53

Confusion Matrix:
[[ 29 102]
 [ 33 124]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.47      0.22      0.30       131
         1.0       0.55      0.79      0.65       157

    accuracy                           0.53       288
   macro avg       0.51      0.51      0.47       288
weighted avg       0.51      0.53      0.49       288


Feature Importances:
                              Feature  Importance
1                         Log_Returns    0.086201
4                       Close_Ratio_5    0.084482
0   weighted compound sentiment score    0.084171
6                      Close_Ratio_60    0.082698
14                    Volatility_1000    0.080547
2                       Close_Ratio_2    0.080439
8                       Volatility_60    0.080136
11                     Volatility_250    0.079682
12                   Close_Ratio_1000    0.079077
9                     Close_Ratio_250    0.075494
10                  

In [51]:
y_test

1602    1.0
1603    1.0
1604    1.0
1605    1.0
1607    0.0
       ... 
1991    1.0
1992    0.0
1993    1.0
1994    0.0
1995    0.0
Name: Target, Length: 288, dtype: float64

In [52]:
# Count unique values in y_test
unique_counts = pd.Series(y_test).value_counts()

# Display the counts
print(unique_counts)

Target
1.0    157
0.0    131
Name: count, dtype: int64


In [53]:
features = ['Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000']
target = ['Target']


import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `df` is your DataFrame
features = features
target = 'Target'

# Split data into training (80%) and testing (20%) sets
split_index = int(len(df) * 0.8)
train_data = df.iloc[:split_index]
test_data = df.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.5  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


Accuracy: 0.57

Confusion Matrix:
[[ 54  77]
 [ 48 109]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.53      0.41      0.46       131
         1.0       0.59      0.69      0.64       157

    accuracy                           0.57       288
   macro avg       0.56      0.55      0.55       288
weighted avg       0.56      0.57      0.56       288


Feature Importances:
             Feature  Importance
3      Close_Ratio_5    0.094885
1      Close_Ratio_2    0.092169
0        Log_Returns    0.090860
13   Volatility_1000    0.088457
11  Close_Ratio_1000    0.087026
10    Volatility_250    0.086158
5     Close_Ratio_60    0.085454
8    Close_Ratio_250    0.084634
7      Volatility_60    0.084008
12        Trend_1000    0.052294
9          Trend_250    0.051850
6           Trend_60    0.051133
4            Trend_5    0.032159
2            Trend_2    0.018912
